In [2]:
import requests
import pandas as pd
import io

In [3]:
def crawler(data_time):
    url =  'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + data_time + '&type=ALLBUT0999'
    page = requests.get(url).text
    use_text = page.splitlines()
    for i, text in enumerate(use_text):
        if text == '"證券代號","證券名稱","成交股數","成交筆數","成交金額","開盤價","最高價","最低價","收盤價","漲跌(+/-)","漲跌價差","最後揭示買價","最後揭示買量","最後揭示賣價","最後揭示賣量","本益比",':
            initial_point = i
            break
    test_df = pd.read_csv(io.StringIO((''.join([text[:-1] + '\n' for text in use_text[initial_point:]]))))
    test_df['證券代號'] = test_df['證券代號'].apply(lambda x:x.replace('"', ''))
    test_df['證券代號'] = test_df['證券代號'].apply(lambda x:x.replace('=', ''))
    return test_df

In [4]:
import datetime
import time

def trans_date(date_time):
    return str(date_time).split(' ')[0].replace('-', '')

def parse_n_days(start_date, n):
    df_dict = {}
    now_date = start_date
    for i in range(n):
        time.sleep(3)
        try:
            df = crawler(trans_date(now_date))
            print('Successful!!' + ' ' + trans_date(now_date))
            df_dict.update({trans_date(now_date):df})
        except:
            print('Fails at' + ' ' + trans_date(now_date))
        now_date = now_date - datetime.timedelta(days=1)
    return df_dict

In [ ]:
result_dict = parse_n_days(datetime.datetime.now(), 600)

In [13]:
for key in result_dict.keys():
    result_dict[key].to_csv(str(key) + '.csv')

### Build a sqlite database, manipulate csv files

In [1]:
import pandas as pd
import sqlite3
import glob

### Read all csv files
- import glob module
- Analyzing Data from Multiple Files

In [2]:
All_csv_file = glob.glob('*.csv')

In [4]:
df = pd.read_csv(All_csv_file[0])

In [ ]:
df.iloc[:,1:].head()  # read from 1st column

### Create a database by using sqlite3 module

In [17]:
dbname = 'TWStock.db'

In [18]:
db = sqlite3.connect(dbname)

In [19]:
%%time
for file_name in All_csv_file:
    pd.read_csv(file_name).iloc[:,1:].to_sql(file_name.replace('.csv', ''), db, if_exists='replace')

CPU times: total: 9.17 s
Wall time: 26.5 s


### Read Database Table

In [20]:
dates_list = [file_name.replace('.csv', '') for file_name in All_csv_file]

In [ ]:
%%time
total_df = pd.DataFrame()
for date in dates_list:
    df = pd.read_sql(con=db, sql='SELECT * FROM "' + date + '"')
    df['Date'] = date
    total_df = total_df.append(df)

In [27]:
total_df.shape

(457457, 18)

In [28]:
dbname_2 = 'TWStock_2'
db2 = sqlite3.connect(dbname_2)

In [30]:
%%time
total_dict = dict(tuple(total_df.groupby('證券代號')))

CPU times: total: 875 ms
Wall time: 875 ms


In [31]:
for key in total_dict.keys():
    df = total_dict[key].iloc[:,2:]
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by=['Date'])
    df.to_sql(key, db2, if_exists='replace')

In [33]:
pd.read_sql(con=db2, sql='SELECT * FROM "2330"').tail()

,index,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Date
395,448,台積電,"24,991,291","26,792","12,569,771,761",506.00,508.00,500.00,504.00,-,5.0,503.00,6,504.00,315,19.81,2022-08-01 00:00:00
396,448,台積電,"42,669,591","63,879","20,973,293,337",494.00,496.00,488.50,492.00,-,12.0,491.50,44,492.00,350,19.34,2022-08-02 00:00:00
397,448,台積電,"29,838,832","25,570","14,823,224,632",494.00,501.00,493.00,501.00,+,9.0,500.00,9,501.00,581,19.69,2022-08-03 00:00:00
398,448,台積電,"26,589,086","27,173","13,279,624,282",499.00,503.00,495.00,500.00,-,1.0,499.50,592,500.00,4,19.65,2022-08-04 00:00:00
399,448,台積電,"35,052,642","49,928","17,966,410,242",509.00,516.00,507.00,516.00,+,16.0,515.00,732,516.00,478,20.28,2022-08-05 00:00:00
